In [1]:
# pqd_predictions_test01_dataset04.ipynb
# This file loads the CNN model trained in pqd_cnn_test01_dataset04, 
# i.e. with the model structure same as the paper and the pqd_dataset_04.
# Then, this file loads the prediction dataset from pqd_dataset_04, and
# generates a CSV output file of the prediction score of each image.

In [3]:
# Import dependencies
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import models, utils, preprocessing
from google.colab import drive, files

In [4]:
# Connect to the Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Load the trained CNN model
model_path = '/content/drive/My Drive/saved_models/pqd_cnn_test01_dataset04_model.h5'
cnn = models.load_model(model_path)

In [6]:
# Import the testing dataset for extracting the categories
testing_dataset = utils.image_dataset_from_directory(
    directory='/content/drive/My Drive/pqd_dataset_04/testing_set',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    image_size=(200, 200)
)

Found 1000 files belonging to 10 classes.


In [12]:
# Extract the categories of the data
type_names = testing_dataset.class_names
df_columns = ["image_name"] + type_names

In [14]:
# Define a pd dataframe
df = pd.DataFrame(
    columns=df_columns
)

In [16]:
# Import the prediction dataset
prediction_set_path = '/content/drive/My Drive/pqd_dataset_04/prediction_set'
prediction_set = tf.io.gfile.listdir(prediction_set_path)

In [54]:
# Iterate through the dataset, and
# make predictions with the trained CNN model.
# Log the prediction scores in the pd dataframe
count = 0
for image_name in prediction_set:
    prediction_image = utils.load_img(
        path=prediction_set_path + "/" + image_name,
        color_mode='grayscale',
        target_size=(200, 200)
    )
    prediction_image_array = preprocessing.image.img_to_array(
        prediction_image
    )
    prediction_image_array = np.array(
        [prediction_image_array]
    )
    predictions = cnn.predict(prediction_image_array)
    df_row = [image_name]
    for prediction in predictions[0]:
      df_row.append(prediction)
    df.loc[count] = df_row
    count += 1

In [55]:
# Sanity check
print(df.head())

                        image_name      flickers     harmonics  interruptions  \
0  swells_harmonics_sample0569.png  1.840084e-17  7.266716e-13   5.502704e-17   
1      interruptions_sample018.png  3.643335e-06  1.297955e-10   9.874240e-01   
2  swells_harmonics_sample0582.png  2.832530e-20  8.176146e-13   2.686468e-14   
3          flickers_sample0659.png  9.999114e-01  3.776266e-28   1.921260e-20   
4             swells_sample096.png  6.140131e-15  7.824053e-20   1.389709e-14   

   interruptions_harmonics  osc_transients          sags  sags_harmonics  \
0             2.169196e-12    1.104628e-07  1.260007e-19    8.525175e-14   
1             1.354514e-04    4.819397e-04  9.513879e-03    2.862627e-08   
2             1.000540e-06    9.774185e-06  2.219847e-19    9.413446e-14   
3             2.030648e-21    8.851486e-05  1.722802e-15    2.640846e-22   
4             2.365805e-15    4.519915e-08  2.096017e-17    1.749445e-17   

         spikes        swells  swells_harmonics  
0  1.3

In [56]:
# Save the pd dataframe as a CSV file.
# Download the CSV file
df.to_csv('pqd_predictions_test01_dataset04.csv') 
files.download('pqd_predictions_test01_dataset04.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>